In [1]:
from multiprocessing.dummy import Pool as ThreadPool
from multiprocessing import Lock, Value  
import time
import requests
from lxml import etree,html as lhtml
from bs4 import BeautifulSoup
import re
import pandas as pd
from tqdm.notebook import tqdm
from lxml import etree, html as lhtml
from time import sleep

In [2]:
link = 'https://www.respublica.ru/authors/'
auth_link_ = list()
with open('authors.txt') as f:
    for num, author in enumerate(f):
        author = re.sub(r'\n', '', author)
        auth_link_.append(link + author +'?page=' )

In [3]:
def get_page(url, n_attempts=5, t_sleep=1, **kwargs):
    r_get = requests.get(url, **kwargs)
    n_attempts -= 1
    
    while n_attempts > 0 and not r_get.ok:
            time.sleep(t_sleep)
            r_get = requests.get(url, **kwargs)
            n_attempts -= 1
    
    if n_attempts >= 0:
        return r_get

In [4]:
def links(author_):
    page = get_page(author_).text
    soup = BeautifulSoup(page, 'html.parser')
    link = list(map(lambda s:s['href'],soup.find_all('a', class_="rd-listing-product-item__image-wrapper")))
    link = ['https://www.respublica.ru'+ i for i in link]
    return link

In [5]:
def all_l(author_):
    book_author_ = list()
    num = 1
    while True:
        page = links(author_ + str(num))
        book_author_.extend(page)
        num = num + 1
        if len(page) == 0:
            break
    return book_author_

In [6]:
%%time
with ThreadPool(processes=10) as pool:
    mass = pool.map(all_l, auth_link_)
pool.join()
all_links = [ element for row in mass for element in row ]
len(all_links)

Wall time: 28.8 s


2453

In [7]:
def process_page(url):
    
    page = get_page(url).text
    tree = lhtml.fromstring(page)
    field={}
    
    #Блок_1
    block1 = tree.xpath('//div[@class="rd-rating-stars"]')[0]

    
    #1). Категория
    category = tree.xpath('//span[@class="rd-page-breadcrumbs-item"]//span[@itemprop="name"]')
    c = ';'.join(ctg.text for ctg in category)
    field['Категория'] = c
        
    #2). Название
    field["Название"] =  tree.xpath('//h1[@class="rd-page-product__title"]')[0].text
    
    #3). Автор
    field["Автор"] = ';'.join(tree.xpath('//a[@itemprop="brand"]//text()'))
    
    #4). ID
    field["ID"] = tree.xpath('//span[@itemprop="sku"]')[0].text
    
    #5). Превью
    preview = tree.xpath('//a[@class="download-pdf"]//attribute::href')
    field['Превью'] = 'https://www.respublica.ru'+preview[0] if len(preview) != 0 else None
    
    #6). Изображение
    img = tree.xpath('//img[@class="rd-page-product__img"]//attribute::src')
    field['Изображение'] = 'https://www.respublica.ru'+img[0]
    
    #7). Оценка
    assessment = block1.xpath('.//attribute::content')
    field['Оценка'] = assessment[3] if len(assessment) !=0 else None
    
    #8). Цена
    value_new = tree.xpath('//span[@class="num"]')[0].text
    field['Цена'] = re.sub(r'\D','',value_new)

    #9). Цена(старая)
    value_old = tree.xpath('//span[@class="prev"]')
    field["Цена (старая)"] = re.sub(r'\D', '', value_old[0].text) if len(value_old) != 0 else None
    
    #10). В наличии
    availability = tree.xpath('//span[@class="current"]')
    field['В наличии'] = True if len(availability) != 0 else False
    
    #11). Описание
    description = tree.xpath('//div[@class="rd-page-product__desc-body"]//text()')
    description = '\n'.join(description)
    field['Описание'] = re.sub(r'\s+', ' ', description.strip())
    
    #1). Число отзывов
    review_num = block1.xpath('.//attribute::content')
    field['Число отзывов'] = review_num[0] if len(review_num) != 0 else None

    #2). Число оценок
    assessment_num = block1.xpath('.//attribute::content')
    field['Число оценок'] = assessment_num[1] if len(assessment_num) != 0 else None
    
    #Блок_2
    block2 = tree.xpath('//div[@class="rd-page-product__desc-params"]')[0]
    
    #Таблица
    name = block2.xpath('.//p/*[contains(@itemprop, "name")]/text()')
    value = block2.xpath('.//p/*[contains(@itemprop, "value")]/text()')
    field.update(zip(name, value))
    return field

In [8]:
mutex = Lock()
n_processed = Value('i',0)
def func_wrapper(uid):
    res = process_page(uid)
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

In [9]:
%%time

with ThreadPool(processes=50) as pool:
    res = pool.map(func_wrapper, all_links)
pool.join()

2450 objects are processed...Wall time: 2min 32s


In [10]:
df_1 = pd.DataFrame(res)

#Если что-то не получилось скачать после всех попыток, то фильтруем
res = [site for site in res if site]

df_2 = pd.DataFrame(res)

In [11]:
print("Загружено страниц: ", df_2.shape[0])
print("Неудачно: ", df_1.shape[0] - df_2.shape[0])

Загружено страниц:  2453
Неудачно:  0


In [12]:
df_2.head()

,Категория,Название,Автор,ID,Превью,Изображение,Оценка,Цена,Цена (старая),В наличии,...,Тип,Вид бумаги,Материал,"Размер, см",Пол,Назначение,Тематика,Упаковка,Рисунок,Иллюстрации
0,Книги; Художественная литература; Фантастика ...,Летать или бояться,Стивен Кинг,529756,https://www.respublica.ru/items/389843/downloa...,https://www.respublica.ru/uploads/00/00/00/d0/...,None,320,480,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Книги; Художественная литература; Фантастика ...,Пост сдал,Стивен Кинг,529754,https://www.respublica.ru/items/389841/downloa...,https://www.respublica.ru/uploads/01/00/00/d0/...,None,190,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Книги; Художественная литература; Фантастика ...,Сияние,Стивен Кинг,528463,None,https://www.respublica.ru/uploads/01/00/00/cx/...,None,390,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Книги; Художественная литература; Фантастика ...,Буря столетия,Стивен Кинг,525197,https://www.respublica.ru/items/385161/downloa...,https://www.respublica.ru/uploads/00/00/00/cq/...,None,200,None,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Книги; Художественная литература; Фантастика ...,Под Куполом. Шестое чувство,Стивен Кинг,521176,None,https://www.respublica.ru/uploads/00/00/00/cg/...,None,260,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df_2.isnull().sum()

Категория                    0
Название                     0
Автор                        0
ID                           0
Превью                    1279
Изображение                  0
Оценка                    2364
Цена                         0
Цена (старая)             2150
В наличии                    0
Описание                     0
Число отзывов             2364
Число оценок              2364
ISBN                       313
Издательство               200
Серия                      803
Обложка                    208
Формат                      19
Количество страниц         210
Год издания                222
Язык                       218
Раздел                     611
Направление               1790
Страна-производитель       602
Вес, г                    1685
Возрастные ограничения    1707
Жанр                      2048
Герои                     2450
Эпоха                     2452
Возраст                   2068
Иллюстратор               2185
Тип                       2245
Вид бума

In [14]:
df_2.to_csv('z.michael_hw_3.csv')